In [1]:
import requests
from datetime import datetime
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix

## Tracking Election Data for 2026's Key Congressional Races

#### According to Roll Call (Altimari 2025), vulnerable seats in the 2026 election of the US House of Representatives include:

**AK-1** Nick Begich III, R
 <br>
**CA-13** Adam Gray, D
<br>
**CA-27** George Whitesides, D
<br>
**CA-45** Derek Tran, D
<br>
**CO-8** Gabe Evans, R
<br>
**IA-1** Mariannette Miller-Meeks, R
<br>
**ME-2** Jared Golden, D
<br>
**NC-1** Don Davis, D
<br>
**NE-2** Don Bacon, R
<br>
**NY-3** Tom Suozzi, D
<br>
**NY-4** Laura Gillen, D
<br>
**NY-17** Mike Lawler, R
<br>
**NY-19** Josh Riley, D
<br>
**TX-28** Henry Cuellar, D
<br>
**TX-34** *(TX-15 pre-2023)* Vicente Gonzalez, D
<br>
**WA-3** Marie Gluesenkamp Pérez, D
<br>
<br>
**Sources**
<br>
Altimari, Daniela. 2025. “An Early Look at House Members Who Could Be Vulnerable in 2026 - Roll Call.” Roll Call. January 7, 2025. https://rollcall.com/2025/01/07/house-vulnerable-2026-midterm-elections/.<br>
“Data | MIT Election Lab.” 2019. Mit.edu. May 29, 2019. https://electionlab.mit.edu/data.
‌

In [3]:
results = pd.read_csv('1976-2022-house.csv')

In [5]:
results.shape

(32452, 20)

In [7]:
results.columns

Index(['year', 'state', 'state_po', 'state_fips', 'state_cen', 'state_ic',
       'office', 'district', 'stage', 'runoff', 'special', 'candidate',
       'party', 'writein', 'mode', 'candidatevotes', 'totalvotes',
       'unofficial', 'version', 'fusion_ticket'],
      dtype='object')

In [9]:
results.office.unique()

array(['US HOUSE'], dtype=object)

In [11]:
results.drop('office', axis = 1, inplace = True)

In [13]:
results.drop('state', axis = 1, inplace = True)

In [15]:
results.drop('state_cen', axis = 1, inplace = True)

In [17]:
results.drop('state_ic', axis = 1, inplace = True)

In [19]:
results.stage.unique()

array(['GEN', 'PRI'], dtype=object)

In [21]:
# Write a function to clean up the party column for more efficient visualization

def clean_party(party):
    if pd.isna(party) or not isinstance(party, str):
        return "NOT LISTED"
    
    # Dictionary for common party name corrections
    party_map = {
        "DEMOCRATIC": "DEMOCRAT",
        "DEMOCRAT": "DEMOCRAT",
        "REPUBLICAN": "REPUBLICAN",
        "INDEPENDENT": "INDEPENDENT",
        "GREEN": "GREEN",
        "LIBERTARIAN": "LIBERTARIAN",
        "SOCIALIST": "BLUE OTHER",
        "D.C. STATEHOOD GREEN": "GREEN",
        'PACIFIC GREEN': 'GREEN',
        'CONSERVATIVE': 'RED OTHER',
        'PROGRESSIVE': 'BLUE OTHER',
        'INDEPENDENT PARTY OF DELAWARE': 'INDEPENDENT',
        'DEMOCRATIC-FARMER-LABOR': 'DEMOCRAT'
    }

    # Check if party is in the mapping dictionary
    for key, value in party_map.items():
        if key in party:
            return value

    return "OTHER"  # Categorize remaining parties under "Other"

# Apply function to 'party' column in a pandas DataFrame
results['party_rev'] = results['party'].apply(clean_party)

results.head()

,year,state_po,state_fips,district,stage,runoff,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version,fusion_ticket,party_rev
0,1976,AL,1,1,GEN,False,False,BILL DAVENPORT,DEMOCRAT,False,TOTAL,58906,157170,False,20230706,False,DEMOCRAT
1,1976,AL,1,1,GEN,False,False,JACK EDWARDS,REPUBLICAN,False,TOTAL,98257,157170,False,20230706,False,REPUBLICAN
2,1976,AL,1,1,GEN,False,False,WRITEIN,NaN,True,TOTAL,7,157170,False,20230706,False,NOT LISTED
3,1976,AL,1,2,GEN,False,False,J CAROLE KEAHEY,DEMOCRAT,False,TOTAL,66288,156362,False,20230706,False,DEMOCRAT
4,1976,AL,1,2,GEN,False,False,"WILLIAM L ""BILL"" DICKINSON",REPUBLICAN,False,TOTAL,90069,156362,False,20230706,False,REPUBLICAN


In [23]:
results.year.dtype

dtype('int64')

In [25]:
# I'm mostly interested in election data since 2016

results = results[results['year'] > 2014]
results.shape

(5615, 17)

In [27]:
# I want a new column to show the percentage of votes each candidate got

results['percentage_votes'] = results['candidatevotes'] / results['totalvotes'] * 100

In [29]:
# Now that I dropped the state column, I'll rename state_po to state

results.rename(columns = {'state_po': 'state'}, inplace = True)

In [31]:
# Reorder columns so they make more sense to me

results = results.iloc[:, [0, 1, 3, 16, 17, 7, 11, 12, 2, 4, 5, 6, 8, 9, 10, 13, 14, 15]]
results.head()

,year,state,district,party_rev,percentage_votes,candidate,candidatevotes,totalvotes,state_fips,stage,runoff,special,party,writein,mode,unofficial,version,fusion_ticket
26837,2016,AL,1,REPUBLICAN,96.382467,BRADLEY BYRNE,208083,215893,1,GEN,NaN,False,REPUBLICAN,False,TOTAL,False,20230706,False
26838,2016,AL,1,NOT LISTED,3.617533,WRITEIN,7810,215893,1,GEN,NaN,False,NaN,True,TOTAL,False,20230706,False
26839,2016,AL,2,REPUBLICAN,48.768548,MARTHA ROBY,134886,276584,1,GEN,NaN,False,REPUBLICAN,False,TOTAL,False,20230706,False
26840,2016,AL,2,DEMOCRAT,40.526205,NATHAN MATHIS,112089,276584,1,GEN,NaN,False,DEMOCRAT,False,TOTAL,False,20230706,False
26841,2016,AL,2,NOT LISTED,10.705247,WRITEIN,29609,276584,1,GEN,NaN,False,NaN,True,TOTAL,False,20230706,False


In [33]:
# I'll check the shape before I download this set for my records

results.shape

(5615, 18)

In [35]:
# I'll save this, might want to do some Tableau work with this dataset

results.to_csv('congress_election_results_2016-2022.csv')

In [37]:
# But I still want to do some data wrangling here.

results['dem_votes'] = ''
results['rep_votes'] = ''
results['dem_cand'] = ''
results['rep_cand'] = ''

In [45]:
# I'll aggregate by year, state, district, and party, then sum the 'candidatevotes'
# Must remember to set index as false-- I often forget this!

aggregated_results = results.groupby(['year', 'state', 'district', 'party_rev'], as_index = False)['candidatevotes'].sum()
aggregated_results.head(20)

,year,state,district,party_rev,candidatevotes
0,2016,AK,0,DEMOCRAT,111019
1,2016,AK,0,LIBERTARIAN,31770
2,2016,AK,0,NOT LISTED,1228
3,2016,AK,0,OTHER,9093
4,2016,AK,0,REPUBLICAN,155088
5,2016,AL,1,NOT LISTED,7810
6,2016,AL,1,REPUBLICAN,208083
7,2016,AL,2,DEMOCRAT,112089
8,2016,AL,2,NOT LISTED,29609
9,2016,AL,2,REPUBLICAN,134886


In [47]:
# Now I'm going to pivot and make the parties columns

pivot_results = aggregated_results.pivot(index = ['year', 'state', 'district'], columns = 'party_rev', values = 'candidatevotes').fillna(0)

In [49]:
# And of course reset the index. I need readability.

pivot_results.reset_index(inplace = True)
pivot_results.head(20)

party_rev,year,state,district,BLUE OTHER,DEMOCRAT,GREEN,INDEPENDENT,LIBERTARIAN,NOT LISTED,OTHER,RED OTHER,REPUBLICAN
0,2016,AK,0,0.0,111019.0,0.0,0.0,31770.0,1228.0,9093.0,0.0,155088.0
1,2016,AL,1,0.0,0.0,0.0,0.0,0.0,7810.0,0.0,0.0,208083.0
2,2016,AL,2,0.0,112089.0,0.0,0.0,0.0,29609.0,0.0,0.0,134886.0
3,2016,AL,3,0.0,94549.0,0.0,0.0,0.0,391.0,0.0,0.0,192164.0
4,2016,AL,4,0.0,0.0,0.0,0.0,0.0,3519.0,0.0,0.0,235925.0
5,2016,AL,5,0.0,102234.0,0.0,0.0,0.0,445.0,0.0,0.0,205647.0
6,2016,AL,6,0.0,83709.0,0.0,0.0,0.0,284.0,0.0,0.0,245313.0
7,2016,AL,7,0.0,229330.0,0.0,0.0,0.0,3698.0,0.0,0.0,0.0
8,2016,AR,1,0.0,0.0,0.0,0.0,57181.0,0.0,0.0,0.0,183866.0
9,2016,AR,2,0.0,111347.0,0.0,0.0,14342.0,303.0,0.0,0.0,176472.0


In [51]:
display(pivot_results.dtypes)

party_rev
year             int64
state           object
district         int64
BLUE OTHER     float64
DEMOCRAT       float64
GREEN          float64
INDEPENDENT    float64
LIBERTARIAN    float64
NOT LISTED     float64
OTHER          float64
RED OTHER      float64
REPUBLICAN     float64
dtype: object

In [53]:
# I think I should make these party columns integers.

pivot_results['BLUE OTHER'] = pivot_results['BLUE OTHER'].astype(int)
pivot_results['RED OTHER'] = pivot_results['RED OTHER'].astype(int)
pivot_results['NOT LISTED'] = pivot_results['NOT LISTED'].astype(int)
pivot_results['DEMOCRAT'] = pivot_results['DEMOCRAT'].astype(int)
pivot_results['GREEN'] = pivot_results['GREEN'].astype(int)
pivot_results['INDEPENDENT'] = pivot_results['INDEPENDENT'].astype(int)
pivot_results['LIBERTARIAN'] = pivot_results['LIBERTARIAN'].astype(int)
pivot_results['OTHER'] = pivot_results['OTHER'].astype(int)
pivot_results['REPUBLICAN'] = pivot_results['REPUBLICAN'].astype(int)
display(pivot_results.dtypes)

party_rev
year            int64
state          object
district        int64
BLUE OTHER      int32
DEMOCRAT        int32
GREEN           int32
INDEPENDENT     int32
LIBERTARIAN     int32
NOT LISTED      int32
OTHER           int32
RED OTHER       int32
REPUBLICAN      int32
dtype: object

In [55]:
# And get a total votes column

pivot_results['TOTAL_VOTES'] = pivot_results['BLUE OTHER'] + pivot_results['RED OTHER'] + pivot_results['OTHER'] + pivot_results['DEMOCRAT'] + pivot_results['REPUBLICAN'] + pivot_results['INDEPENDENT'] + pivot_results['GREEN'] + pivot_results['LIBERTARIAN'] + pivot_results['NOT LISTED']
pivot_results.head(20)

party_rev,year,state,district,BLUE OTHER,DEMOCRAT,GREEN,INDEPENDENT,LIBERTARIAN,NOT LISTED,OTHER,RED OTHER,REPUBLICAN,TOTAL_VOTES
0,2016,AK,0,0,111019,0,0,31770,1228,9093,0,155088,308198
1,2016,AL,1,0,0,0,0,0,7810,0,0,208083,215893
2,2016,AL,2,0,112089,0,0,0,29609,0,0,134886,276584
3,2016,AL,3,0,94549,0,0,0,391,0,0,192164,287104
4,2016,AL,4,0,0,0,0,0,3519,0,0,235925,239444
5,2016,AL,5,0,102234,0,0,0,445,0,0,205647,308326
6,2016,AL,6,0,83709,0,0,0,284,0,0,245313,329306
7,2016,AL,7,0,229330,0,0,0,3698,0,0,0,233028
8,2016,AR,1,0,0,0,0,57181,0,0,0,183866,241047
9,2016,AR,2,0,111347,0,0,14342,303,0,0,176472,302464


In [57]:
pivot_results['BLUE_VOTES'] = pivot_results['BLUE OTHER'] + pivot_results['DEMOCRAT'] 
pivot_results['RED_VOTES'] = pivot_results['RED OTHER'] + pivot_results['REPUBLICAN'] 
pivot_results.head(20)

party_rev,year,state,district,BLUE OTHER,DEMOCRAT,GREEN,INDEPENDENT,LIBERTARIAN,NOT LISTED,OTHER,RED OTHER,REPUBLICAN,TOTAL_VOTES,BLUE_VOTES,RED_VOTES
0,2016,AK,0,0,111019,0,0,31770,1228,9093,0,155088,308198,111019,155088
1,2016,AL,1,0,0,0,0,0,7810,0,0,208083,215893,0,208083
2,2016,AL,2,0,112089,0,0,0,29609,0,0,134886,276584,112089,134886
3,2016,AL,3,0,94549,0,0,0,391,0,0,192164,287104,94549,192164
4,2016,AL,4,0,0,0,0,0,3519,0,0,235925,239444,0,235925
5,2016,AL,5,0,102234,0,0,0,445,0,0,205647,308326,102234,205647
6,2016,AL,6,0,83709,0,0,0,284,0,0,245313,329306,83709,245313
7,2016,AL,7,0,229330,0,0,0,3698,0,0,0,233028,229330,0
8,2016,AR,1,0,0,0,0,57181,0,0,0,183866,241047,0,183866
9,2016,AR,2,0,111347,0,0,14342,303,0,0,176472,302464,111347,176472


In [59]:
# Mmk I can get rid of these individual party columns now

pivot_results.drop('BLUE OTHER', axis = 1, inplace = True)

In [61]:
pivot_results.drop('RED OTHER', axis = 1, inplace = True)

In [63]:
pivot_results.drop('DEMOCRAT', axis = 1, inplace = True)

In [65]:
pivot_results.drop('REPUBLICAN', axis = 1, inplace = True)

In [67]:
# Gonna smush these two into one 'other' catchall

pivot_results['OTHER2'] = pivot_results['OTHER'] + pivot_results['NOT LISTED']

In [69]:
# And drop its components

pivot_results.drop('OTHER', axis = 1, inplace = True)

In [71]:
pivot_results.drop('NOT LISTED', axis = 1, inplace = True)

In [73]:
# Rename my new column, no second banana.

pivot_results.rename(columns = {'OTHER2': 'OTHER'}, inplace = True)

In [75]:
# Aaaaaand how are we lookin'

pivot_results.head()

party_rev,year,state,district,GREEN,INDEPENDENT,LIBERTARIAN,TOTAL_VOTES,BLUE_VOTES,RED_VOTES,OTHER
0,2016,AK,0,0,0,31770,308198,111019,155088,10321
1,2016,AL,1,0,0,0,215893,0,208083,7810
2,2016,AL,2,0,0,0,276584,112089,134886,29609
3,2016,AL,3,0,0,0,287104,94549,192164,391
4,2016,AL,4,0,0,0,239444,0,235925,3519


In [77]:
# Mmk not bad. I wanna combine the state and district number though. I'll start by making the district column a string.

pivot_results['district'] = pivot_results['district'].astype(str)

In [79]:
# Now I'll create a new column where I'll concatenate the existing district and state columns.

pivot_results['full_district'] = pivot_results['state'] + '-' + pivot_results['district']

# And check how we're lookin'
pivot_results.head()

party_rev,year,state,district,GREEN,INDEPENDENT,LIBERTARIAN,TOTAL_VOTES,BLUE_VOTES,RED_VOTES,OTHER,full_district
0,2016,AK,0,0,0,31770,308198,111019,155088,10321,AK-0
1,2016,AL,1,0,0,0,215893,0,208083,7810,AL-1
2,2016,AL,2,0,0,0,276584,112089,134886,29609,AL-2
3,2016,AL,3,0,0,0,287104,94549,192164,391,AL-3
4,2016,AL,4,0,0,0,239444,0,235925,3519,AL-4


In [81]:
# I love the way that new column looks. But these all caps are aggressive.

pivot_results.rename(columns = {'GREEN': 'green', 'INDEPENDENT': 'independent', 'LIBERTARIAN': 'libertarian', 'TOTAL_VOTES': 'total_votes', 'BLUE_VOTES': 'blue_votes', 'RED_VOTES': 'red_votes', 'OTHER': 'other_votes'}, inplace = True)

In [85]:
# And get the percentage of each party's votes

pivot_results['blue_pct'] = pivot_results['blue_votes'] / pivot_results['total_votes']
pivot_results['red_pct'] = pivot_results['red_votes'] / pivot_results['total_votes']
pivot_results['other_pct'] = pivot_results['other_votes'] / pivot_results['total_votes']
pivot_results['green_pct'] = pivot_results['green'] / pivot_results['total_votes']
pivot_results['libertarian_pct'] = pivot_results['libertarian'] / pivot_results['total_votes']
pivot_results['ind_pct'] = pivot_results['independent'] / pivot_results['total_votes']

# Drumroll, please...
pivot_results.head()

party_rev,year,state,district,green,independent,libertarian,total_votes,blue_votes,red_votes,other_votes,full_district,blue_pct,red_pct,other_pct,green_pct,libertarian_pct,ind_pct
0,2016,AK,0,0,0,31770,308198,111019,155088,10321,AK-0,0.360220,0.503209,0.033488,0.0,0.103083,0.0
1,2016,AL,1,0,0,0,215893,0,208083,7810,AL-1,0.000000,0.963825,0.036175,0.0,0.000000,0.0
2,2016,AL,2,0,0,0,276584,112089,134886,29609,AL-2,0.405262,0.487685,0.107052,0.0,0.000000,0.0
3,2016,AL,3,0,0,0,287104,94549,192164,391,AL-3,0.329320,0.669318,0.001362,0.0,0.000000,0.0
4,2016,AL,4,0,0,0,239444,0,235925,3519,AL-4,0.000000,0.985303,0.014697,0.0,0.000000,0.0


In [89]:
# Reorder columns

pivot_results = pivot_results.iloc[:, [0, 10, 6, 11, 12, 13, 16, 15, 14, 7, 8, 9, 4, 3, 5, 1, 2]]
pivot_results.head()

party_rev,year,full_district,total_votes,blue_pct,red_pct,other_pct,ind_pct,libertarian_pct,green_pct,blue_votes,red_votes,other_votes,independent,green,libertarian,state,district
0,2016,AK-0,308198,0.360220,0.503209,0.033488,0.0,0.103083,0.0,111019,155088,10321,0,0,31770,AK,0
1,2016,AL-1,215893,0.000000,0.963825,0.036175,0.0,0.000000,0.0,0,208083,7810,0,0,0,AL,1
2,2016,AL-2,276584,0.405262,0.487685,0.107052,0.0,0.000000,0.0,112089,134886,29609,0,0,0,AL,2
3,2016,AL-3,287104,0.329320,0.669318,0.001362,0.0,0.000000,0.0,94549,192164,391,0,0,0,AL,3
4,2016,AL-4,239444,0.000000,0.985303,0.014697,0.0,0.000000,0.0,0,235925,3519,0,0,0,AL,4


In [91]:
# Oh, right, that decimal looks a lil funky

pivot_results['blue_pct'] = pivot_results['blue_pct'] * 100
pivot_results['red_pct'] = pivot_results['red_pct'] * 100
pivot_results['other_pct'] = pivot_results['other_pct'] * 100
pivot_results['ind_pct'] = pivot_results['ind_pct'] * 100
pivot_results['libertarian_pct'] = pivot_results['libertarian_pct'] * 100
pivot_results['green_pct'] = pivot_results['green_pct'] * 100

pivot_results.head()

party_rev,year,full_district,total_votes,blue_pct,red_pct,other_pct,ind_pct,libertarian_pct,green_pct,blue_votes,red_votes,other_votes,independent,green,libertarian,state,district
0,2016,AK-0,308198,36.021973,50.320898,3.348821,0.0,10.308308,0.0,111019,155088,10321,0,0,31770,AK,0
1,2016,AL-1,215893,0.000000,96.382467,3.617533,0.0,0.000000,0.0,0,208083,7810,0,0,0,AL,1
2,2016,AL-2,276584,40.526205,48.768548,10.705247,0.0,0.000000,0.0,112089,134886,29609,0,0,0,AL,2
3,2016,AL-3,287104,32.931969,66.931844,0.136188,0.0,0.000000,0.0,94549,192164,391,0,0,0,AL,3
4,2016,AL-4,239444,0.000000,98.530345,1.469655,0.0,0.000000,0.0,0,235925,3519,0,0,0,AL,4


In [93]:
# Ugh OK one more reorder

pivot_results = pivot_results.iloc[:, [0, 1, 2, 3, 4, 5, 6, 8, 7, 9, 10, 11, 12, 13, 14, 15, 16]]

pivot_results.head()

party_rev,year,full_district,total_votes,blue_pct,red_pct,other_pct,ind_pct,green_pct,libertarian_pct,blue_votes,red_votes,other_votes,independent,green,libertarian,state,district
0,2016,AK-0,308198,36.021973,50.320898,3.348821,0.0,0.0,10.308308,111019,155088,10321,0,0,31770,AK,0
1,2016,AL-1,215893,0.000000,96.382467,3.617533,0.0,0.0,0.000000,0,208083,7810,0,0,0,AL,1
2,2016,AL-2,276584,40.526205,48.768548,10.705247,0.0,0.0,0.000000,112089,134886,29609,0,0,0,AL,2
3,2016,AL-3,287104,32.931969,66.931844,0.136188,0.0,0.0,0.000000,94549,192164,391,0,0,0,AL,3
4,2016,AL-4,239444,0.000000,98.530345,1.469655,0.0,0.0,0.000000,0,235925,3519,0,0,0,AL,4


In [95]:
# Now I want to ID the winner for each row

pivot_results['winning_party'] = pivot_results.iloc[:, 3:9].idxmax(axis = 1)
pivot_results.head()

C:\Users\natal\AppData\Local\Temp\ipykernel_18512\1694798089.py:3: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  pivot_results['winning_party'] = pivot_results.iloc[:, 3:9].idxmax(axis = 1)


party_rev,year,full_district,total_votes,blue_pct,red_pct,other_pct,ind_pct,green_pct,libertarian_pct,blue_votes,red_votes,other_votes,independent,green,libertarian,state,district,winning_party
0,2016,AK-0,308198,36.021973,50.320898,3.348821,0.0,0.0,10.308308,111019,155088,10321,0,0,31770,AK,0,red_pct
1,2016,AL-1,215893,0.000000,96.382467,3.617533,0.0,0.0,0.000000,0,208083,7810,0,0,0,AL,1,red_pct
2,2016,AL-2,276584,40.526205,48.768548,10.705247,0.0,0.0,0.000000,112089,134886,29609,0,0,0,AL,2,red_pct
3,2016,AL-3,287104,32.931969,66.931844,0.136188,0.0,0.0,0.000000,94549,192164,391,0,0,0,AL,3,red_pct
4,2016,AL-4,239444,0.000000,98.530345,1.469655,0.0,0.0,0.000000,0,235925,3519,0,0,0,AL,4,red_pct


In [97]:
# I need to check a random sample

pivot_results.sample(10)

party_rev,year,full_district,total_votes,blue_pct,red_pct,other_pct,ind_pct,green_pct,libertarian_pct,blue_votes,red_votes,other_votes,independent,green,libertarian,state,district,winning_party
1288,2020,WA-5,404360,38.514443,61.285735,0.199822,0.000000,0.0,0.00000,155737,247815,808,0,0,0,WA,5,red_pct
835,2018,UT-4,269271,50.121996,49.864263,0.013741,0.000000,0.0,0.00000,134964,134270,37,0,0,0,UT,4,blue_pct
1658,2022,TN-2,207762,32.091046,67.908954,0.000000,0.000000,0.0,0.00000,66673,141089,0,0,0,0,TN,2,red_pct
1630,2022,PA-1,367380,45.132832,54.867168,0.000000,0.000000,0.0,0.00000,165809,201571,0,0,0,0,PA,1,red_pct
859,2018,WI-2,317295,97.422273,0.000000,2.577727,0.000000,0.0,0.00000,309116,0,8179,0,0,0,WI,2,blue_pct
1605,2022,OH-2,257862,25.496196,74.503804,0.000000,0.000000,0.0,0.00000,65745,192117,0,0,0,0,OH,2,red_pct
1607,2022,OH-4,290156,30.805153,69.194847,0.000000,0.000000,0.0,0.00000,89383,200773,0,0,0,0,OH,4,red_pct
1553,2022,ND-0,238586,0.000000,62.199375,0.227591,37.573034,0.0,0.00000,0,148399,543,89644,0,0,ND,0,red_pct
562,2018,GA-14,229724,23.498198,76.501802,0.000000,0.000000,0.0,0.00000,53981,175743,0,0,0,0,GA,14,red_pct
1046,2020,LA-3,340120,29.482242,67.764318,0.000000,0.000000,0.0,2.75344,100275,230480,0,0,0,9365,LA,3,red_pct


In [99]:
# Column looks good, but I see more reordering I want to do

pivot_results = pivot_results.iloc[:, [0, 1, 17, 2, 3, 4, 5, 6, 8, 7, 9, 10, 11, 12, 13, 14, 15, 16]]

In [101]:
# And I just want the winning party column to say blue or red (or ind, green, etc., I don't want the "_pct")

pivot_results['winning_party'].replace('blue_pct', 'blue', inplace = True)

C:\Users\natal\AppData\Local\Temp\ipykernel_18512\3364361188.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pivot_results['winning_party'].replace('blue_pct', 'blue', inplace = True)


In [103]:
pivot_results['winning_party'].replace('red_pct', 'red', inplace = True)

C:\Users\natal\AppData\Local\Temp\ipykernel_18512\4179778561.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pivot_results['winning_party'].replace('red_pct', 'red', inplace = True)


In [105]:
pivot_results['winning_party'].replace('ind_pct', 'ind', inplace = True)

C:\Users\natal\AppData\Local\Temp\ipykernel_18512\405389803.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pivot_results['winning_party'].replace('ind_pct', 'ind', inplace = True)


In [107]:
pivot_results['winning_party'].replace('other_pct', 'other', inplace = True)

C:\Users\natal\AppData\Local\Temp\ipykernel_18512\1532707388.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pivot_results['winning_party'].replace('other_pct', 'other', inplace = True)


In [109]:
pivot_results.shape

(1741, 18)

In [111]:
# party_rev has become my index, so I'll rename accordingly

pivot_results.rename(columns = {'party_rev': 'idx'}, inplace = True)
pivot_results.head()

party_rev,year,full_district,winning_party,total_votes,blue_pct,red_pct,other_pct,ind_pct,libertarian_pct,green_pct,blue_votes,red_votes,other_votes,independent,green,libertarian,state,district
0,2016,AK-0,red,308198,36.021973,50.320898,3.348821,0.0,10.308308,0.0,111019,155088,10321,0,0,31770,AK,0
1,2016,AL-1,red,215893,0.000000,96.382467,3.617533,0.0,0.000000,0.0,0,208083,7810,0,0,0,AL,1
2,2016,AL-2,red,276584,40.526205,48.768548,10.705247,0.0,0.000000,0.0,112089,134886,29609,0,0,0,AL,2
3,2016,AL-3,red,287104,32.931969,66.931844,0.136188,0.0,0.000000,0.0,94549,192164,391,0,0,0,AL,3
4,2016,AL-4,red,239444,0.000000,98.530345,1.469655,0.0,0.000000,0.0,0,235925,3519,0,0,0,AL,4


In [380]:
# And see what I can do with this in Tableau!

pivot_results.to_csv('pivot_results.csv')